# ¿Que color pintarán el coche?

Una empresa dedicada a la venta de coches usados se enfrenta al desafío de determinar el color óptimo para repintar vehículos que llegan en condiciones deficientes. Tras
evaluar las opciones, decide limitarse a los colores blanco y negro, por ser los más comunes en el mercado. 

Para decidir el color de repintado de cada coche, la empresa planea desarrollar un modelo predictivo que, basándose en las características de los vehículos en el mercado de segunda mano, determine si originalmente eran blancos o negros. 

La base de datos disponible incluye las siguientes variables independientes 
- Precio de venta, 
- Cantidad de Impuestos a pagar, 
- Fabricante, 
- Año de fabricación, 
- Categoría, 
- Interior de cuero, 
- Tipo de combustible, 
- Volumen del motor, 
- Kilometraje, 
- Cilindros, 
- Tipo de caja de cambios, 
- Ruedas motrices, 
- Lugar del volante, 
- Número de Airbags

Y de la variable dependiente 
- Color. 

La decisión final es si el coche debe pintarse de blanco o no.

## Inicialización de datos

In [1]:
# Librerias

import pandas as pd
import re
import numpy as np
from category_encoders import OneHotEncoder


In [2]:
# Cargar los datos

df = pd.read_excel('datasets/datos_tarea25.xlsx')
display(df.head())

,Price,Levy,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags
0,39493,891,HYUNDAI,2016,Jeep,Yes,Diesel,2,160931 km,4,Automatic,Front,Left wheel,White,4
1,1803,761,TOYOTA,2010,Hatchback,Yes,Hybrid,1.8,258909 km,4,Automatic,Front,Left wheel,White,12
2,1098,394,TOYOTA,2014,Sedan,Yes,Hybrid,2.5,398069 km,4,Automatic,Front,Left wheel,Black,12
3,941,1053,MERCEDES-BENZ,2014,Sedan,Yes,Diesel,3.5,184467 km,6,Automatic,Rear,Left wheel,White,12
4,1019,1055,LEXUS,2013,Jeep,Yes,Hybrid,3.5,138038 km,6,Automatic,Front,Left wheel,White,12


## Análisis de datos

In [3]:
# Revisamos información sobre el dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Price             4340 non-null   int64 
 1   Levy              4340 non-null   object
 2   Manufacturer      4340 non-null   object
 3   Prod. year        4340 non-null   int64 
 4   Category          4340 non-null   object
 5   Leather interior  4340 non-null   object
 6   Fuel type         4340 non-null   object
 7   Engine volume     4340 non-null   object
 8   Mileage           4340 non-null   object
 9   Cylinders         4340 non-null   int64 
 10  Gear box type     4340 non-null   object
 11  Drive wheels      4340 non-null   object
 12  Wheel             4340 non-null   object
 13  Color             4340 non-null   object
 14  Airbags           4340 non-null   int64 
dtypes: int64(4), object(11)
memory usage: 508.7+ KB


No observamos *valores ausentes* en los registros.

In [4]:
# Revisión de duplicados
dupli = df.duplicated().sum()
print(f'Hay {dupli} registros duplicados de {len(df)} registros totales')
print(f'... el {((dupli / len(df))*100):.2f} % de registros duplicados')

Hay 1534 registros duplicados de 4340 registros totales
... el 35.35 % de registros duplicados


Eliminamos los *registros duplicados* encontrados:

In [5]:
# Eliminación de duplicados
df = df.drop_duplicates()
dupli = df.duplicated().sum()
print(f'Ahora hay {dupli} registros duplicados. Quedan {len(df)} registros')


Ahora hay 0 registros duplicados. Quedan 2806 registros


Renombramos columnas:

In [6]:
# Funtion "to_snake_case" 
def to_snake_case(name):
    # Reemplaza espacios en blanco por guion bajo
    name = re.sub(r'\s+', '_', name)
    # Elimina caracteres especiales excepto letras, números y guion bajo
    name = re.sub(r'[^A-Za-z0-9_]', '', name)
    # Convierte CamelCase/PascalCase a snake_case
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    snake = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    return snake

df = df.rename(columns = lambda x: to_snake_case(x))

df.columns


Index(['price', 'levy', 'manufacturer', 'prod_year', 'category',
       'leather_interior', 'fuel_type', 'engine_volume', 'mileage',
       'cylinders', 'gear_box_type', 'drive_wheels', 'wheel', 'color',
       'airbags'],
      dtype='object')

In [7]:
df.head()


,price,levy,manufacturer,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,wheel,color,airbags
0,39493,891,HYUNDAI,2016,Jeep,Yes,Diesel,2,160931 km,4,Automatic,Front,Left wheel,White,4
1,1803,761,TOYOTA,2010,Hatchback,Yes,Hybrid,1.8,258909 km,4,Automatic,Front,Left wheel,White,12
2,1098,394,TOYOTA,2014,Sedan,Yes,Hybrid,2.5,398069 km,4,Automatic,Front,Left wheel,Black,12
3,941,1053,MERCEDES-BENZ,2014,Sedan,Yes,Diesel,3.5,184467 km,6,Automatic,Rear,Left wheel,White,12
4,1019,1055,LEXUS,2013,Jeep,Yes,Hybrid,3.5,138038 km,6,Automatic,Front,Left wheel,White,12


Tipos de datos:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 0 to 4339
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             2806 non-null   int64 
 1   levy              2806 non-null   object
 2   manufacturer      2806 non-null   object
 3   prod_year         2806 non-null   int64 
 4   category          2806 non-null   object
 5   leather_interior  2806 non-null   object
 6   fuel_type         2806 non-null   object
 7   engine_volume     2806 non-null   object
 8   mileage           2806 non-null   object
 9   cylinders         2806 non-null   int64 
 10  gear_box_type     2806 non-null   object
 11  drive_wheels      2806 non-null   object
 12  wheel             2806 non-null   object
 13  color             2806 non-null   object
 14  airbags           2806 non-null   int64 
dtypes: int64(4), object(11)
memory usage: 350.8+ KB


In [9]:
# Revisemos los valores / categorías en columnas de tipo object

columnas_object = df.select_dtypes(include='object').columns.tolist()

for column in columnas_object:
    print('column', column, ', unique values ', len(df[column].unique()))
    print(df[column].sort_values().unique())
    print()

    

column levy , unique values  203
['-' '1017' '1018' '1024' '1047' '1051' '1053' '1055' '1058' '1076' '1077'
 '1079' '1091' '1099' '1104' '1111' '1163' '11706' '11714' '1172' '1174'
 '1178' '1185' '1197' '1202' '1205' '1211' '1236' '1249' '1266' '1267'
 '1268' '1273' '1275' '1277' '1292' '1296' '1323' '1324' '1325' '1327'
 '1342' '1356' '1360' '1363' '1375' '1384' '1391' '1399' '1405' '1426'
 '1481' '1493' '1503' '1505' '1514' '1525' '1528' '1537' '1575' '1598'
 '1604' '1624' '1646' '1659' '1664' '1694' '1714' '1723' '175' '1750'
 '1793' '1811' '1817' '1820' '1823' '1845' '1848' '1850' '1935' '1946'
 '1951' '1968' '2002' '2018' '2056' '2070' '2108' '2148' '2151' '2188'
 '2223' '2225' '2265' '2266' '2297' '2364' '2410' '2418' '2455' '2570'
 '2705' '271' '2736' '2858' '2959' '3057' '308' '345' '3505' '353' '3571'
 '382' '394' '397' '420' '431' '441' '442' '456' '462' '463' '475' '501'
 '502' '503' '517' '518' '528' '530' '531' '552' '579' '583' '584' '585'
 '586' '595' '603' '605' '607' '

In [10]:
# Columna 'levy', cambiar a tipo entero
df['levy'] = pd.to_numeric(df['levy'], errors='coerce').astype('Int64')

In [11]:
df.isna().sum()

price                 0
levy                634
manufacturer          0
prod_year             0
category              0
leather_interior      0
fuel_type             0
engine_volume         0
mileage               0
cylinders             0
gear_box_type         0
drive_wheels          0
wheel                 0
color                 0
airbags               0
dtype: int64

Podemos observar que han surgido valores NaN en 'levy', entonces vamos a imputarlos.

In [12]:
# Imputar valores NaN en 'levy' usando la mediana de coches con la misma category, prod_year, manufacturer y gear_box_type.
# Si no hay coincidencias, usar la mediana de coches con precio similar (+/- 5%).

def imputar_levy(row):
    if pd.isna(row['levy']):
        # Primer filtro: coincidencia exacta en 4 columnas
        filtro = (
            (df['category'] == row['category']) &
            (df['prod_year'] == row['prod_year']) &
            (df['manufacturer'] == row['manufacturer']) &
            (df['gear_box_type'] == row['gear_box_type']) &
            (~df['levy'].isna())
        )
        median_levy = df.loc[filtro, 'levy'].median()
        if not pd.isna(median_levy):
            return median_levy

        # Segundo filtro: coches con precio similar (+/- 5%)
        precio = row['price']
        margen = precio * 0.05
        filtro_precio = (
            (df['price'] >= precio - margen) &
            (df['price'] <= precio + margen) &
            (~df['levy'].isna())
        )
        median_levy_precio = df.loc[filtro_precio, 'levy'].median()
        if not pd.isna(median_levy_precio):
            return median_levy_precio

        # Tercer filtro: mediana general
        median_general = df['levy'].median()
        if not pd.isna(median_general):
            return median_general

        # Si todo falla, devuelve 0
        return 0
    else:
        return row['levy']

df['levy'] = df.apply(imputar_levy, axis=1)
df['levy'] = df['levy'].astype(int)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 0 to 4339
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             2806 non-null   int64 
 1   levy              2806 non-null   int64 
 2   manufacturer      2806 non-null   object
 3   prod_year         2806 non-null   int64 
 4   category          2806 non-null   object
 5   leather_interior  2806 non-null   object
 6   fuel_type         2806 non-null   object
 7   engine_volume     2806 non-null   object
 8   mileage           2806 non-null   object
 9   cylinders         2806 non-null   int64 
 10  gear_box_type     2806 non-null   object
 11  drive_wheels      2806 non-null   object
 12  wheel             2806 non-null   object
 13  color             2806 non-null   object
 14  airbags           2806 non-null   int64 
dtypes: int64(5), object(10)
memory usage: 350.8+ KB


In [14]:
# Columna 'leather_interior' cambio a tipo de dato binario
df['leather_interior'] = df['leather_interior'].map({'Yes': 1, 'No':0}) 
df['leather_interior'].unique()

array([1, 0])

In [15]:
df[df['engine_volume'].str.contains('Turbo', case=False, na=False)].head(5)

,price,levy,manufacturer,prod_year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,wheel,color,airbags
10,20385,3571,MERCEDES-BENZ,2006,Sedan,1,Diesel,2.2 Turbo,210000 km,4,Tiptronic,Rear,Left wheel,Black,8
43,72130,1079,LEXUS,2018,Jeep,1,Petrol,2.0 Turbo,18918 km,4,Tiptronic,Front,Left wheel,Black,6
81,31361,639,HYUNDAI,2014,Sedan,1,Petrol,2.0 Turbo,62000 km,4,Tiptronic,Front,Left wheel,White,12
101,84675,1358,MERCEDES-BENZ,2018,Sedan,1,Petrol,2.0 Turbo,4500 km,6,Tiptronic,Rear,Left wheel,Black,12
119,21953,585,HYUNDAI,2013,Sedan,0,Hybrid,2.4 Turbo,20000 km,2,Tiptronic,Front,Right-hand drive,Black,4


In [16]:
# Crear columna 'turbo' a partir de 'engine_volume' y eliminar 'Turbo' de 'engine_volume' y convertir a tipo float
df.insert(df.columns.get_loc('engine_volume') + 1, 'turbo', df['engine_volume'].str.contains('Turbo', case=False, na=False).astype(int))
df['engine_volume'] = df['engine_volume'].str.replace('Turbo', '', case=False, regex=False).str.strip()
df['engine_volume'] = df['engine_volume'].astype(float)

In [17]:
df[df['turbo'] == 1].head()

,price,levy,manufacturer,prod_year,category,leather_interior,fuel_type,engine_volume,turbo,mileage,cylinders,gear_box_type,drive_wheels,wheel,color,airbags
10,20385,3571,MERCEDES-BENZ,2006,Sedan,1,Diesel,2.2,1,210000 km,4,Tiptronic,Rear,Left wheel,Black,8
43,72130,1079,LEXUS,2018,Jeep,1,Petrol,2.0,1,18918 km,4,Tiptronic,Front,Left wheel,Black,6
81,31361,639,HYUNDAI,2014,Sedan,1,Petrol,2.0,1,62000 km,4,Tiptronic,Front,Left wheel,White,12
101,84675,1358,MERCEDES-BENZ,2018,Sedan,1,Petrol,2.0,1,4500 km,6,Tiptronic,Rear,Left wheel,Black,12
119,21953,585,HYUNDAI,2013,Sedan,0,Hybrid,2.4,1,20000 km,2,Tiptronic,Front,Right-hand drive,Black,4


In [18]:
# Revisamos la columna "mileage"
df.mileage.unique()

array(['160931 km', '258909 km', '398069 km', ..., '180234 km',
       '132700 km', '186923 km'], dtype=object)

In [19]:
# Columna mileage, eliminar la palabra " km" y convertir a tipo entero

df.mileage = df.mileage.str.replace(' km', '', regex=False)
df.mileage = pd.to_numeric(df.mileage, errors='coerce')
df.mileage.unique()


array([160931, 258909, 398069, ..., 180234, 132700, 186923])

In [20]:
df.head()

,price,levy,manufacturer,prod_year,category,leather_interior,fuel_type,engine_volume,turbo,mileage,cylinders,gear_box_type,drive_wheels,wheel,color,airbags
0,39493,891,HYUNDAI,2016,Jeep,1,Diesel,2.0,0,160931,4,Automatic,Front,Left wheel,White,4
1,1803,761,TOYOTA,2010,Hatchback,1,Hybrid,1.8,0,258909,4,Automatic,Front,Left wheel,White,12
2,1098,394,TOYOTA,2014,Sedan,1,Hybrid,2.5,0,398069,4,Automatic,Front,Left wheel,Black,12
3,941,1053,MERCEDES-BENZ,2014,Sedan,1,Diesel,3.5,0,184467,6,Automatic,Rear,Left wheel,White,12
4,1019,1055,LEXUS,2013,Jeep,1,Hybrid,3.5,0,138038,6,Automatic,Front,Left wheel,White,12


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 0 to 4339
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             2806 non-null   int64  
 1   levy              2806 non-null   int64  
 2   manufacturer      2806 non-null   object 
 3   prod_year         2806 non-null   int64  
 4   category          2806 non-null   object 
 5   leather_interior  2806 non-null   int64  
 6   fuel_type         2806 non-null   object 
 7   engine_volume     2806 non-null   float64
 8   turbo             2806 non-null   int64  
 9   mileage           2806 non-null   int64  
 10  cylinders         2806 non-null   int64  
 11  gear_box_type     2806 non-null   object 
 12  drive_wheels      2806 non-null   object 
 13  wheel             2806 non-null   object 
 14  color             2806 non-null   object 
 15  airbags           2806 non-null   int64  
dtypes: float64(1), int64(8), object(7)
memory usage

In [22]:
# Revisemos las columnas de tipo object que podrían convertirse a tipos numéricos o categóricos
print("Columnas tipo object:", columnas_object)
print()

for col in columnas_object:
    print(f"Columna: {col}")
    print(df[col].unique()[:10])  # Muestra los primeros 10 valores únicos
    print()

# Sugerencias:
# Convertir columnas seleccionadas a tipo 'category'

Columnas tipo object: ['levy', 'manufacturer', 'category', 'leather_interior', 'fuel_type', 'engine_volume', 'mileage', 'gear_box_type', 'drive_wheels', 'wheel', 'color']

Columna: levy
[ 891  761  394 1053 1055 1079  810 1850 1249 3571]

Columna: manufacturer
['HYUNDAI' 'TOYOTA' 'MERCEDES-BENZ' 'LEXUS']

Columna: category
['Jeep' 'Hatchback' 'Sedan']

Columna: leather_interior
[1 0]

Columna: fuel_type
['Diesel' 'Hybrid' 'Petrol']

Columna: engine_volume
[2.  1.8 2.5 3.5 2.2 4.7 1.5 3.3 2.4 1.6]

Columna: mileage
[160931 258909 398069 184467 138038  76000  74146  54317 364523  39709]

Columna: gear_box_type
['Automatic' 'Tiptronic']

Columna: drive_wheels
['Front' 'Rear' '4x4']

Columna: wheel
['Left wheel' 'Right-hand drive']

Columna: color
['White' 'Black']



In [23]:
categorical_cols = ['manufacturer', 'category', 'fuel_type', 'gear_box_type', 'drive_wheels', 'wheel', 'color']
df[categorical_cols] = df[categorical_cols].astype('category')
print(df[categorical_cols].dtypes)

manufacturer     category
category         category
fuel_type        category
gear_box_type    category
drive_wheels     category
wheel            category
color            category
dtype: object


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2806 entries, 0 to 4339
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   price             2806 non-null   int64   
 1   levy              2806 non-null   int64   
 2   manufacturer      2806 non-null   category
 3   prod_year         2806 non-null   int64   
 4   category          2806 non-null   category
 5   leather_interior  2806 non-null   int64   
 6   fuel_type         2806 non-null   category
 7   engine_volume     2806 non-null   float64 
 8   turbo             2806 non-null   int64   
 9   mileage           2806 non-null   int64   
 10  cylinders         2806 non-null   int64   
 11  gear_box_type     2806 non-null   category
 12  drive_wheels      2806 non-null   category
 13  wheel             2806 non-null   category
 14  color             2806 non-null   category
 15  airbags           2806 non-null   int64   
dtypes: category(7), float64(1), i

# Procesamiento de datos

## Codificaciòn de variables categóricas

In [29]:
# Aplicar One-Hot Encoder a las columnas categóricas seleccionadas

# Seleccion de columnas categoricas


col_cat = df.select_dtypes(include=['category']).columns.tolist()


# Apply OneHotEncoder for categorical features

ohe = OneHotEncoder(use_cat_names=True)
# No es necesario volver a seleccionar columnas, col_cat ya está definido correctamente
col_ohe = ohe.fit_transform(df[col_cat])

# Transform to dataframe the OHE features
df_ohe = pd.DataFrame(data=col_ohe, columns=ohe.get_feature_names_out()).astype(int)

# Remove categorical columns "cat_col" in features
df.drop(labels=col_cat, axis=1, inplace=True)

# Add OHE features transformed into "X" features
df = df.merge(right=df_ohe, left_index=True, right_index=True)
df

,price,levy,prod_year,leather_interior,engine_volume,turbo,mileage,cylinders,airbags,manufacturer_HYUNDAI,...,fuel_type_Petrol,gear_box_type_Automatic,gear_box_type_Tiptronic,drive_wheels_Front,drive_wheels_Rear,drive_wheels_4x4,wheel_Left wheel,wheel_Right-hand drive,color_White,color_Black
0,39493,891,2016,1,2.0,0,160931,4,4,1,...,0,1,0,1,0,0,1,0,1,0
1,1803,761,2010,1,1.8,0,258909,4,12,0,...,0,1,0,1,0,0,1,0,1,0
2,1098,394,2014,1,2.5,0,398069,4,12,0,...,0,1,0,1,0,0,1,0,0,1
3,941,1053,2014,1,3.5,0,184467,6,12,0,...,0,1,0,0,1,0,1,0,1,0
4,1019,1055,2013,1,3.5,0,138038,6,12,0,...,0,1,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332,22197,503,2012,1,1.5,0,50000,4,4,0,...,1,1,0,1,0,0,1,0,0,1
4334,1176,986,2010,1,2.4,0,129170,4,12,0,...,0,1,0,1,0,0,1,0,0,1
4335,470,831,2017,1,1.6,0,25635,4,12,1,...,1,1,0,1,0,0,1,0,0,1
4336,26017,1624,2009,1,3.5,0,132700,4,4,0,...,1,1,0,1,0,0,1,0,1,0
